In [6]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well.

I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS tool designed to streamline SOC 2 compliance and enhance your audit preparation with the power of AI. 

In today’s regulatory landscape, ensuring compliance can be complex and time-consuming. Our platform automates key processes, helping you identify gaps, manage documentation, and stay ahead of audit requirements—all while significantly reducing the burden on your team.

Key features include:
- Automated readiness assessments
- Real-time compliance tracking
- Customizable reporting capabilities

Companies like [relevant client or industry example] have experienced improvements in both efficiency and audit outcomes, allowing them to focus on what truly matters—growing their business.

I’d love to schedule a brief call to discuss how ComplAI can help [Recipient's Company] streamline your compliance processe